### 确保电脑里有GPU
- `!nvidia-smi`：可以看到GPU的使用率（查看**显卡信息**）
- 在pytorch中，**每个数组都有一个设备**（device），通常称其为环境（context）
- 所有深度学习框架默认所有变量和相关的计算都分配给CPU——**要指定去GPU上**

In [39]:
!nvidia-smi

Mon Oct 28 09:55:41 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX150      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   51C    P0    N/A /  N/A |    505MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 计算设备

In [5]:
import torch

from torch import nn

In [8]:
torch.device('cpu'), torch.cuda.device('cuda') #, torch.cuda.device('cuda:1')

(device(type='cpu'), <torch.cuda.device at 0x2ee3e11b9d0>)

**查询可用GPU的数量**

In [2]:
torch.cuda.device_count()

1

In [4]:
print(torch.cuda.is_available())

True


In [3]:
print(torch.version.cuda)

11.1


以下两个函数**允许在请求的GPU不存在的情况下运行代码**
- `torch.cuda.device('cuda')`
- `torch.device(f'cuda:{i}')`

In [32]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        # return torch.cuda.device('cuda')
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gups():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())
    ]
    return devices if devices else [torch.decive('cpu')]

In [33]:
print(try_gpu(),'\n', try_gpu(3), '\n', try_all_gups())

cuda:0 
 cpu 
 [device(type='cuda', index=0)]


### 张量和GPU

查询**张量所在的设备**，默认情况下，张量是在CPU上创建的

In [34]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

### 存储在GPU上
- 一般来说，需要保证不创建超过GPU显存限制的数据
- `device=try_gpu(i)`，其中`i`可以填成GPU的index数

In [41]:
x = torch.ones(2, 3, device=try_gpu())
y = torch.rand(2, 3, device=try_gpu())
# y = torch.rand(2, 3, device=try_gpu(1))
x , y

(tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'),
 tensor([[0.0615, 0.2499, 0.6369],
         [0.7280, 0.2063, 0.2898]], device='cuda:0'))

#### 运算涉及的变量必须在**同一个GPU上**才可以在GPU上运算
- GPU数据挪到CPU是**很慢**的——**出于性能的考虑**
- **在设备（CPU、GPU和其他机器）之间传递数据比计算慢得多**——使得并行化变得更加困难

In [42]:
# z = y.cuda(0)
# x + z
x + y

tensor([[1.0615, 1.2499, 1.6369],
        [1.7280, 1.2063, 1.2898]], device='cuda:0')

### 神经网络和GPU
- `net.to(device=try_gpu())`——`Module`**只能**用`to(device=)`

In [43]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(x)

tensor([[0.7511],
        [0.7511]], device='cuda:0', grad_fn=<AddmmBackward>)

**确认模型、参数存储在同一GPU上**

In [44]:
net[0].weight.device

device(type='cuda', index=0)

#### 在GPU上做运算的**办法**：
将数据挪到GPU上，则对应的操作也会在GPU上完成
- 需要手动copy到GPU上，若有多个GPU，需要保证数据在同一GPU上做运行
- 对于神经网络也是一样，需要将权重copy到GPU上，输入也copy到GPU上，就可以在GPU上forward，backward也会在同一GPU上做运算

- data preprocessing在GPU上不一定支持的比较好
- 在深度学习中，当在GPU上计算数据（比如张量）后，如果想要**在Python中打印该张量的值或将其转换为NumPy数组进行进一步处理**——需要将数据从**GPU内存**复制到**CPU内存**
    - 这一过程涉及数据传输，会带来**额外的传输时间开销**
    - **全局解释器锁（GIL）**：Python的解释器使用这个机制，确保只有一个线程可以同时执行Python字节码
        - 会导致串行化的效果，尤其是多线程环境下，因为Python会阻塞其他线程，直到当前线程完成
        - 这种串行化对GPU到CPU的传输操作影响很大，因为所有相关的代码和线程都必须等待GIL释放
        - 当频繁把**数据传到CPU并在Python环境中记录**（如转换为`ndarray`或打印输出），每次数据传输操作都会触发GIL
    - **更好的实践**：避免频繁、细粒度的GPU到CPU传输
        - 如：把日志记录的内存分配在GPU上，等一段时间或累积一定量后，再把这些数据批量传输到CPU，从而减少传输次数和对GIL的干扰

### 小结

* 我们可以指定用于存储和计算的设备，例如CPU或GPU。**默认情况下**，数据在主内存中创建，然后使用CPU进行计算。
* 深度学习框架要求计算的**所有输入数据都在同一设备上**，无论是CPU还是GPU。
* 不经意地移动数据可能会**显著降低性能**。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy `ndarray`中）时，将触发全局解释器锁，从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。

### 练习
1. 尝试一个计算量更大的任务，比如大矩阵的乘法，看看CPU和GPU之间的速度差异。再试一个计算量很小的任务呢？
- GPU很快啊
    - `round(..., n)`：对计算结果保留两位小数

In [51]:
import time
import torch

# 计算量较大的任务
X = torch.rand((10000, 10000))
Y = X.cuda(0)
time_start = time.time()
Z = torch.mm(X, X)
time_end = time.time()
print(f'larger cup time cost:{round((time_end - time_start) * 1000)}ms') 

time_start = time.time()
Z = torch.mm(Y, Y)
time_end = time.time()
print(f'larger gpu time cost:{round((time_end - time_start) * 1000)}ms') 

# 计算很小的任务
X = torch.rand((100, 100))
Y = X.cuda()
Z = torch.mm(X, X)
time_end = time.time()
print(f'smaller cup time cost:{round((time_end - time_start) * 1000)}ms') 

time_start = time.time()
Z = torch.mm(Y, Y)
time_end = time.time()
print(f'smaller gpu time cost:{round((time_end - time_start) * 1000)}ms') 

larger cup time cost:13253ms
larger gpu time cost:60ms
smaller cup time cost:15225ms
smaller gpu time cost:0ms


2. 我们应该如何在GPU上读写模型参数？
- 使用`net.to(device)`将模型迁移到GPU上，然后在按照之前的方法读写参数
- `map_location=device`的作用：指定在加载模型时，**将模型加载到指定的设备上**`torch.load('', map_location= )`

In [54]:
import torch

from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = MLP()
net.to(device=device)
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[ 0.0689, -0.0808, -0.1696,  ..., -0.0320,  0.0182,  0.1520],
                      [ 0.1008,  0.0257,  0.0150,  ...,  0.1943, -0.1240,  0.1649],
                      [ 0.1503,  0.1013, -0.1000,  ..., -0.2010,  0.2142, -0.0440],
                      ...,
                      [-0.0238,  0.0241, -0.2109,  ..., -0.0222, -0.1948, -0.1119],
                      [ 0.1238,  0.1051, -0.0006,  ..., -0.0684, -0.1344, -0.0890],
                      [-0.1737,  0.1602, -0.0364,  ..., -0.1696, -0.2107, -0.1506]],
                     device='cuda:0')),
             ('hidden.bias',
              tensor([ 2.0553e-01, -3.9608e-02, -2.0917e-02, -2.5621e-02,  1.7343e-01,
                       5.3473e-02,  9.8627e-02,  6.6556e-02,  1.1647e-01,  2.0590e-01,
                       1.7302e-01, -1.8309e-01, -1.3410e-01, -1.4530e-01,  1.2473e-02,
                       1.3898e-02, -2.1442e-02,  1.7967e-02,  1.3954e-01,  1.8899e-01,
                    

In [56]:
X = torch.rand(5, 20).to(device)
output = net(X)

torch.save(net.state_dict(), 'net_params.pth')
net.load_state_dict(torch.load('net_params.pth', map_location=device))

# 从GPU读取到CPU
cpu_params = {name: param.cpu() for name, param in net.named_parameters()}

3. 测量计算1000个$100 \times 100$矩阵的矩阵乘法所需的时间，并记录输出矩阵的Frobenius范数，一次记录一个结果，而不是在GPU上保存日志并仅传输最终结果。（本质是做一个比较）
- 实验1：仅记录1000次$100\times100$次矩阵相乘做用的时间，不需要打印Frobenius范数
- 实验2：记录1000次$100\times100$次矩阵相乘，并每次计算一次就打印Frobenius范数的时间

In [60]:
import time
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

matrics = [torch.rand((100, 100)).to(device) for _ in range(1000)]

# 实验一
start_time_1 = time.time()
for i in range(1000):
    result = torch.mm(matrics[i], matrics[i].t()) # 不转置也无所谓
    forbenius_norm = torch.norm(result)    
end_time_1 = time.time()
print('Time taken:', end_time_1 - start_time_1)


# 实验二
start_time_2 = time.time()
for i in range(1000):
    result = torch.mm(matrics[i], matrics[i])
    forbenius_norm = torch.norm(result)
    print(forbenius_norm)
end_time_2 = time.time()
print('Time taken:', end_time_2 - start_time_2)

print(f'实验一消耗时间：{end_time_1 - start_time_1}，实验二消耗时间：{end_time_2 - start_time_2}')

Time taken: 0.04256129264831543
tensor(2526.0168, device='cuda:0')
tensor(2495.0190, device='cuda:0')
tensor(2502.0378, device='cuda:0')
tensor(2500.8809, device='cuda:0')
tensor(2518.0093, device='cuda:0')
tensor(2534.0884, device='cuda:0')
tensor(2496.5613, device='cuda:0')
tensor(2511.5728, device='cuda:0')
tensor(2523.3743, device='cuda:0')
tensor(2495.8501, device='cuda:0')
tensor(2465.7292, device='cuda:0')
tensor(2521.1113, device='cuda:0')
tensor(2536.7524, device='cuda:0')
tensor(2475.3669, device='cuda:0')
tensor(2524.8977, device='cuda:0')
tensor(2474.9875, device='cuda:0')
tensor(2543.5774, device='cuda:0')
tensor(2518.3130, device='cuda:0')
tensor(2541.7900, device='cuda:0')
tensor(2565.8616, device='cuda:0')
tensor(2510.6245, device='cuda:0')
tensor(2472.4509, device='cuda:0')
tensor(2496.3140, device='cuda:0')
tensor(2539.9141, device='cuda:0')
tensor(2485.4585, device='cuda:0')
tensor(2495.1040, device='cuda:0')
tensor(2516.9851, device='cuda:0')
tensor(2536.8887, devic

4. 测量同时在两个GPU上执行两个矩阵乘法与在一个GPU上按顺序执行两个矩阵乘法所需的时间。提示：应该看到近乎线性的缩放。
- 执行两个矩阵乘法并行在两个GPU上所需要的时间 通常**小的多**比 在单个GPU上岸顺序执行两个矩阵

In [11]:
import torch
import time

a = torch.randn(1000, 1000).cuda()
b = torch.randn(1000, 1000).cuda()

start_time = time.time()
torch.mm(a, b)
torch.mm(a, b)
end_time = time.time()
sequential_time =end_time - start_time
print(f'{sequential_time * 1000}ms')

0.0ms


`torch.cuda.synchronize()`常用于需要精确测量GPU计算时间的场合
- 频繁调用`torch.cuda.synchronize()`会导致性能下降，尤其是不必要时

In [ ]:
import torch
import time

if torch.cuda.device_count() < 2:
    print('需要至少2个GPU来运行此代码')
else:
    size = (1000, 1000)
    
# 串行
device0 = torch.device('cude:0')
a1 = torch.rand(size, device=device0)
b1 = torch.rand(size, device=device0)

start_time = time.time()
c1 = torch.matmul(a1, b1)
torch.cude.synchronize(device0)

c2 = torch.matmul(a1, b1)
torch.cude.synchronize(device0)
sequential_time = time.time() - start_time


# 并行
device1 = torch.device('cuda:1')
A2 = torch.rand(size, device=device0)
B2 = torch.rand(size, device=device0)
A3 = torch.rand(size, device=device1)
B3 = torch.rand(size, device=device1)

start_time = time.time()
C3 = torch.matmul(A2, B2)
C4 = torch.matmul(A3, B3)
torch.cuda.synchronize(device0)
torch.cuda.synchronize(device1)
parallel_time = time.time() - start_time

print(f"双 GPU 并行执行时间: {parallel_time:.4f} 秒")
print(f"速度提升比率（近乎线性）：{speedup:.2f}")

In [12]:
!nvidia-smi

Mon Oct 28 15:12:54 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX150      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P0    N/A /  N/A |    580MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### **删除变量，释放内存**

In [7]:
import gc
del a, b
gc.collect()
torch.cuda.empty_cache()